### Real Case Study



### Instance generation

In [1]:
from src.witnessproblem import RealisticCaseInstanceGenerator
import random

random.seed(1234)

STATIC_WITNESSES = 25
GRID_SIZE = 9
TESTIMONIES_PER_LYING_WITNESS = 5
FILENAME = 'realCase'
BASE_SIMILARITIES = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
LYING_WITNESSES = [0,1,2,3,4,5]

NUMBER_OF_INSTANCES = len(BASE_SIMILARITIES) * len(LYING_WITNESSES)

with open(f"instances/{FILENAME}.txt",'w') as instance_file, open(f"solutions/{FILENAME}.txt",'w') as solution_file:
    instance_string = str(NUMBER_OF_INSTANCES) + '\n'
    route_string = ''
    for base_similarity in BASE_SIMILARITIES:
        for lying_witnesses in LYING_WITNESSES:
            instance_generator = RealisticCaseInstanceGenerator(
                base_similarity=base_similarity, 
                grid_size=GRID_SIZE, 
                lying_witnesses=lying_witnesses,
                static_witnesses=STATIC_WITNESSES,
                testimonies_per_lying_witness=TESTIMONIES_PER_LYING_WITNESS)
            instance, actor_route = instance_generator.generateInstance()

            instance_string += instance.writeToFile()
            route_string += str(base_similarity) + ' ' + str(lying_witnesses) + '\n' + actor_route.writeToFile()
        
    instance_file.write(instance_string)
    solution_file.write(route_string)

In [2]:
from src.geneticAlgorithm.geneticAlgorithm import GeneticAlgorithm
from src.solutions.greedy import GreedyDistance, GreedyTestimonies, PersistentGreedyAlgorithm

RUNS_PER_INSTANCE = 10
RUN_ONCE = 1

genetic_algorithms = [
    (PersistentGreedyAlgorithm(GreedyDistance()), RUN_ONCE),
    (GreedyTestimonies(), RUNS_PER_INSTANCE),
    (GeneticAlgorithm(
        populationSize=150,
        numGenerations = 100,
        mutationProbability=0.651,
        crossoverRate=0.8321,
        localSearchProb=lambda x,y: 0
        ), RUNS_PER_INSTANCE),
    (GeneticAlgorithm(
        populationSize=150,
        numGenerations = 100,
        mutationProbability=0.7194,
        crossoverRate=0.7344,
        localSearchProb=lambda step,totalSteps: step/totalSteps * 0.2104
        ), RUNS_PER_INSTANCE)]



In [3]:
import src.geneticAlgorithm.algorithmTestUtils as tst

corners = [0, GRID_SIZE-1, GRID_SIZE**2-1, GRID_SIZE**2-GRID_SIZE]

tst.executeAlgorithmsRealCaseToCSV(genetic_algorithms, FILENAME, end_vertices=corners)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Starting Execution at 11:16
Calculated the initial population
----Finished instance (0.0; 0) at 11:16 with means [25.0, 23.4, 25.0, 25.0] and max [25, 25, 25, 25]
Calculated the initial population
----Finished instance (0.0; 1) at 11:17 with means [22.0, 19.0, 23.4, 23.5] and max [22, 20, 24, 24]
Calculated the initial population
----Finished instance (0.0; 2) at 11:17 with means [20.0, 18.4, 22.6, 22.5] and max [20, 19, 23, 23]
Calculated the initial population
----Finished instance (0.0; 3) at

In [14]:
experiments_filename = "realCase"

ROWS = len(BASE_SIMILARITIES)
COLUMNS = len(LYING_WITNESSES)
ALGORITHMS = ['Time-based', 'Testimony-based', 'GA', 'MA']

bold_string, bold_string_end = "$\\bf{", "}$"

with open(f"solutions/{experiments_filename}.csv") as f:
    f.readline()


    for algorithm_name in ALGORITHMS:

        print(algorithm_name)

        line = f.readline().split(',')
        INSTANCES = (len(line) - 1)//5

        algorithm_mean_hits_sum = 0

        x,y = 0,0
        print(f"$s={BASE_SIMILARITIES[x]}$", end='')
        for i in range(1, len(line), 5):
            mean = float(line[i])
            maximum = float(line[i+1])
            mean_hits = float(line[i+2])
            real_fitness = float(line[i+3])
            time = float(line[i+4])

            algorithm_mean_hits_sum += mean_hits

            equal_than_original = real_fitness < mean
            print(f" & {bold_string if equal_than_original else ''}{mean_hits}{bold_string_end if equal_than_original else ''}", end='')

            y += 1
            if y >= COLUMNS:
                y = 0
                print(' \\\\')
                x += 1
                if x < ROWS:
                    print(f"$s={BASE_SIMILARITIES[x]}$", end='')
                
        print(f"MEAN HITS: {algorithm_mean_hits_sum/36}")
        print('------------------------------')


Time-based
$s=0$ & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 & 1.0 \\
$s=0.1$ & 1.0 & 1.0 & 0.0 & 1.0 & 0.0 & 0.0 \\
$s=0.2$ & 1.0 & 1.0 & 1.0 & 1.0 & 1.0 & 0.0 \\
$s=0.3$ & 0.0 & 1.0 & 0.0 & 1.0 & 0.0 & $\bf{0.0}$ \\
$s=0.4$ & 0.0 & 0.0 & 1.0 & $\bf{0.0}$ & 0.0 & 0.0 \\
$s=0.5$ & 1.0 & 0.0 & 0.0 & 0.0 & 1.0 & 0.0 \\
MEAN HITS: 0.4444444444444444
------------------------------
Testimony-based
$s=0$ & 0.5 & 0.0 & 0.0 & 0.6 & 0.3 & 0.6 \\
$s=0.1$ & 0.5 & 1.0 & 0.4 & 0.5 & 0.0 & 0.2 \\
$s=0.2$ & 1.0 & 0.7 & 0.6 & 0.1 & 0.3 & 0.0 \\
$s=0.3$ & 0.7 & 0.8 & 0.0 & 0.9 & 0.0 & $\bf{0.5}$ \\
$s=0.4$ & 0.0 & 0.0 & 0.6 & 0.0 & 0.1 & 0.0 \\
$s=0.5$ & 1.0 & 0.0 & 0.7 & 0.5 & 0.7 & 0.0 \\
MEAN HITS: 0.3833333333333333
------------------------------
GA
$s=0$ & 0.8 & 0.5 & 0.5 & 1.0 & $\bf{1.0}$ & 1.0 \\
$s=0.1$ & 1.0 & $\bf{1.0}$ & 0.2 & 0.7 & 0.7 & 0.9 \\
$s=0.2$ & 1.0 & 0.9 & $\bf{1.0}$ & 1.0 & 1.0 & 0.1 \\
$s=0.3$ & 1.0 & 0.9 & 0.6 & 1.0 & 0.9 & $\bf{0.7}$ \\
$s=0.4$ & 0.3 & $\bf{0.2}$ & 0.0 & $\bf{0.1}$ & $\bf{